In [1]:
import os
import cv2
import shapely.geometry as sgeo 
import numpy as np
import json
import glob
from tqdm import tqdm

In [2]:
class Bbox():
    def __init__(self, cx, cy, w, h, rotation=0):
        # center position : cx, cy
        # bbox width and heigh: w, h
        # rotation： positive-> clockwise； negative -> anticlockwise
        self.cx = cx
        self.cy = cy
        self.w = w
        self.h = h
        self.rotation = rotation
        self._init_corner()

    def _init_corner(self):
        rotation = ang2rad(self.rotation)
        ## rotation matrix R
        # cos -sin
        # sin cos
        # opencv is based on the bottom-left [-w/2, h/2]
        w_2 = self.w/2
        h_2 = self.h/2
        w_2cos = w_2*np.cos(rotation)
        w_2sin = w_2*np.sin(rotation)
        h_2cos = h_2*np.cos(rotation)
        h_2sin = h_2*np.sin(rotation)

        self.bottomLeft = [self.cx - w_2cos - h_2sin, self.cy - w_2sin + h_2cos] # R@[-w/2, h/2].t()
        self.topLeft = [self.cx - w_2cos + h_2sin, self.cy - h_2cos - w_2sin] # R@[-w/2, -h/2].t() 
        self.topRight = [2*self.cx - self.bottomLeft[0], 2*self.cy - self.bottomLeft[1]]
        self.bottomRight = [2*self.cx - self.topLeft[0], 2*self.cy - self.topLeft[1]]
        

    def iou(self, target_bbox):
        a = sgeo.Polygon([self.topLeft, self.topRight, self.bottomRight, self.bottomLeft])
        b = sgeo.Polygon([target_bbox.topLeft, target_bbox.topRight, target_bbox.bottomRight, target_bbox.bottomLeft])
        iou = a.intersection(b).area / a.union(b).area
        return iou
    
    def todict(self):
        return {"cx": self.cx, "cy": self.cy,
                "w": self.w, "h": self.h,
                "rotation": self.rotation}

    def tolist_xywh(self):
        return (self.topLeft[0], self.topLeft[1], self.w, self.h)

    def tolist(self):
        return (self.cx, self.cy, self.w, self.h, self.rotation)

def x1y1wh2bbox(bbox_list):
    rotation = bbox_list[4] if len(bbox_list) == 5 else 0
    cx = bbox_list[0] + bbox_list[2] * 0.5
    cy = bbox_list[1] + bbox_list[3] * 0.5
    return Bbox(cx, cy, bbox_list[2], bbox_list[3], rotation)

def ang2rad(a):
    return a/180*np.pi

In [3]:
def plot_bbox(img, bbox, color=(255, 0, 0), size = 10):
        # according to the characteristic of 360, it is impossible to cross top and bottom, thus only consider cross left and right
        #print("plot_bbox", bbox.todict(), bbox.topLeft, bbox.topRight, bbox.topRight, bbox.bottomLeft )
        img_h, img_w = img.shape[:2]
        topleft = bbox.topLeft.copy()
        topRight = bbox.topRight.copy()
        bottomRight = bbox.bottomRight.copy()
        bottomLeft = bbox.bottomLeft.copy()

        topLine = sgeo.LineString([topleft, topRight])
        rightLine = sgeo.LineString([topRight, bottomRight])
        bottomLine = sgeo.LineString([bottomRight, bottomLeft])
        leftLine = sgeo.LineString([bottomLeft, topleft])

        leftBorder = sgeo.LineString([(-1, 0), (-1, img_h)])
        rightBorder = sgeo.LineString([(img_w, 0), (img_w, img_h)])

        lines = []

        if topLine.intersects(leftBorder):
            point = topLine.intersection(leftBorder)
            if isinstance(point, sgeo.Point):
                lines.append([topleft, [img_w-1, point.y]])
                lines.append([[0, point.y], topRight])
        elif topLine.intersects(rightBorder):
            point = topLine.intersection(rightBorder)
            if isinstance(point, sgeo.Point):
                lines.append([topleft, [img_w-1, point.y]])
                lines.append([[0, point.y], topRight])
        else:
            lines.append([topleft, topRight])

        
        if bottomLine.intersects(leftBorder):
            point = bottomLine.intersection(leftBorder)
            if isinstance(point, sgeo.Point):
                lines.append([bottomLeft, [img_w-1, point.y]])
                lines.append([[0, point.y], bottomRight])
        elif bottomLine.intersects(rightBorder):
            point = bottomLine.intersection(rightBorder)
            if isinstance(point, sgeo.Point):
                lines.append([bottomLeft, [img_w-1, point.y]])
                lines.append([[0, point.y], bottomRight])
        else:
            lines.append([bottomLeft, bottomRight])


        if rightLine.intersects(leftBorder):
            point = rightLine.intersection(leftBorder)
            if isinstance(point, sgeo.Point):
                if topRight[0] < 0:
                    lines.append([topRight, [img_w-1, point.y]])
                    lines.append([[0, point.y], bottomRight])
                else:
                    lines.append([topRight, [0, point.y]])
                    lines.append([[img_w-1, point.y], bottomRight])

        elif rightLine.intersects(rightBorder):
            point = rightLine.intersection(rightBorder)
            if isinstance(point, sgeo.Point):
                if topRight[0] < img_w:
                    lines.append([topRight, [img_w-1, point.y]])
                    lines.append([[0, point.y], bottomRight])
                else:
                    lines.append([topRight, [0, point.y]])
                    lines.append([[img_w-1, point.y], bottomRight])
        else:
            lines.append([topRight, bottomRight])


        if leftLine.intersects(leftBorder):
            point = leftLine.intersection(leftBorder)
            if isinstance(point, sgeo.Point):
                if topleft[0] < 0:
                    lines.append([topleft, [img_w-1, point.y]])
                    lines.append([[0, point.y], bottomLeft])
                else:
                    lines.append([topleft, [0, point.y]])
                    lines.append([[img_w-1, point.y], bottomLeft])

        elif leftLine.intersects(rightBorder):
            point = leftLine.intersection(rightBorder)
            if isinstance(point, sgeo.Point):
                if topleft[0] < img_w:
                    lines.append([topleft, [img_w-1, point.y]])
                    lines.append([[0, point.y], bottomLeft])
                else:
                    lines.append([topleft, [0, point.y]])
                    lines.append([[img_w-1, point.y], bottomLeft])
        else:
            lines.append([topleft, bottomLeft])

        for line in lines:
            start, end = line.copy()
            start = np.intp(start)
            end = np.intp(end)
            start[0] %= img_w
            end[0] %= img_w
            cv2.line(img, start, end, color, size)
        return img

In [4]:
def dict2Bbox(bbox_dict):
    return Bbox(bbox_dict["cx"], bbox_dict["cy"], bbox_dict["w"], bbox_dict["h"], bbox_dict["rotation"])

def plot_anno(vis_sequence, images, results_dict, gt_dict, fourcc, img_w = 3840, img_h = 1920 , save_path = '/Users/aarsh/Desktop/CGVI/Thesis/outputs/bbox', tracker = "aiatrack360"):
    """
    Input
    vis_sequence:[str] the visualized sequence
    trackers: [list] the compared trackers
    results_dict: [dict] 
    gt_dict: [dict]
    """

    video_file = os.path.join(save_path, vis_sequence+tracker+".mp4")
    video_writer = cv2.VideoWriter(video_file, fourcc, 15.0, (img_w, img_h))


    anno_classes = results_dict.keys() if len(results_dict) > 0 else gt_dict.keys()
    font_size = 20

    for i, image_name in enumerate(tqdm(images)):
        img = cv2.imread(image_name)
        
        for anno_class in anno_classes:
            gt_anno, gt_anno_absent  = gt_dict[anno_class][vis_sequence]
            if gt_anno_absent[i] < 1:
                continue
            if anno_class == "bbox":
                anno = x1y1wh2bbox(gt_anno[i])
                plot_bbox(img, anno, color=(0, 255, 0), size=font_size)
                

            #start_point = [40, 490, 1190, 1545, 2120, 2450, 3040, 3520]
           
            anno_pred = results_dict[anno_class]['aiatrack'][vis_sequence][i]
            bbox = x1y1wh2bbox(anno_pred)
            plot_bbox(img, bbox, color=(255, 0, 0), size=font_size)
        video_writer.write(img)
    video_writer.release()


def load_gt(sequence, target_anno_file):
        gt_bbox_dict = {}
        gt_bbox = []
        with open(target_anno_file, "r") as output:
            target_annos = json.load(output)
        gt_bbox_absent = np.ones(len(target_annos))
        for i, index in enumerate(target_annos):
            annotation = target_annos[index]
            bbox = annotation["bbox"]
            bbox = dict2Bbox(bbox)
            
            gt_bbox.append(bbox.tolist_xywh())

            if bbox.w ==0 or bbox.h == 0:
                gt_bbox_absent[i] = 0

        gt_bbox_dict.update({sequence: (np.array(gt_bbox), gt_bbox_absent)})
        return gt_bbox_dict

def loadResult(sequence, traj_file,  tracker = 'aiatrack'):
    total_results ={}
    assert os.path.exists(traj_file)
    with open(traj_file, 'r') as f :
        result = {}
        pred_traj = []
        for x in f.readlines():
            anno = x.strip().split(',')             
            if len(anno)<2:
                anno = x.strip().split()  #re.split('; |, ', x)
            #print(bbox)
            pred_traj.append(list(map(float, anno)))
    result.update({sequence: np.array(pred_traj)})
    total_results.update({tracker: result})
   
    return total_results


def main(sequence_num: str, tracker = "AiATrack-360"):
    images = sorted(glob.glob(f'/Users/aarsh/Desktop/CGVI/Thesis/Data/Test_3840x1920/{sequence_num}/image/*.jpg'))
    result_file =f'/Users/aarsh/Desktop/CGVI/Thesis/benchmark/360VOT-bbox-results/{tracker}/{sequence_num}.txt'
    gt_file = f'/Users/aarsh/Desktop/CGVI/Thesis/Data/Test_3840x1920/{sequence_num}/label.json'    
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    img_w = 3840
    img_h = 1920

    gt_bbox_dict = load_gt(sequence_num, gt_file)
    gt_dict={"bbox": gt_bbox_dict}
    results_dict={}
    
    results = loadResult(sequence_num, result_file)
    results_dict={"bbox": results}

    plot_anno(sequence_num, images, results_dict, gt_dict, fourcc, img_w, img_h, tracker=tracker)

Change data here



In [5]:
seq_num = '0117'
main(seq_num, tracker = 'AiATrack-360')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/aarsh/Desktop/CGVI/Thesis/Data/Test_3840x1920/00117/label.json'